# Part 1

Let's import required libraries and objects

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

Retrive the URL & Create BeautifulSoup object

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html=requests.get(url).text

soup=BeautifulSoup(html,'html.parser')


Extract data from Wikipedia and store it in a list

In [3]:
# create empty list
table_contents=[]
# find table
table=soup.find('table')
# find table data and assign to cell(dictionary)
for row in table.findAll('td'):
    cell={}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode']=row.p.text[:3]
        cell['Borough']=(row.span.text).split('(')[0]
        cell['Neighborhood']=(((((row.span.text).split('(')[1]).strip(')')).replace('/',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
    

Assign data into a pandas DataFrame

In [4]:
# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough'] = df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                     'East TorontBusiness reply mail Processing Centre969 Eastern ':'East Toronto Business',
                                    'EtobicokeNorthwest':' EtobiCoke Northwest',
                                     'East YorkEast Toronto':'East York/East Toronto',
                                     'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [5]:
# View 1st  12 rows of the dataframe
df.head(12)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern , Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill , Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [6]:
df.shape

(103, 3)

# Part 2

In [7]:
# read the csv file that contain geospatial coordinates of the Toronto City.
df_coor=pd.read_csv('Geospatial_Coordinates.csv')
df_coor

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [9]:
df=df.rename(columns={'PostalCode':'Postal Code'})
df

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,..."


In [10]:
# list of neighborhoods and their respective Borough,Postal code,longitude and Lattitude of Canada.
# merge previous two dataframe based on Postal Code commonn column
neighborhoods_CA=pd.merge(df,df_coor,on='Postal Code')
neighborhoods_CA

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,...",43.636258,-79.498509


# Part 3

Get the geographical coordinates of Toronto

In [12]:
from geopy import Nominatim
Address='Toronto,Canada'
geolocator=Nominatim(user_agent='Toronto explorer')
location=geolocator.geocode(Address)
longitude=location.longitude
lattitude=location.latitude
print('The Geographical coordinates of Toronto is {},{}'.format(longitude,lattitude))

The Geographical coordinates of Toronto is -79.3839347,43.6534817


In [14]:
CLIENT_ID='PVFHMYLB5SNS10OVXLYOKRKT0YFJWLCZUSUH0CARQO4QMB3N'
CLIENT_SECRET='TROKPKIXHV2EQJUDC32KDANASYMXNZ1MZMLHC30H5TWU3SKL'
VERSION='20210705'
LIMIT=100
print('My foursquare CLIENT_ID:{}'.format(CLIENT_ID))
print('My foursquare CLIENT_SECRET: {}'.format(CLIENT_SECRET))

My foursquare CLIENT_ID:PVFHMYLB5SNS10OVXLYOKRKT0YFJWLCZUSUH0CARQO4QMB3N
My foursquare CLIENT_SECRET: TROKPKIXHV2EQJUDC32KDANASYMXNZ1MZMLHC30H5TWU3SKL


Let's get first neighborhood in Toronto

In [15]:
neighborhoods_CA.loc[1,'Neighborhood']

'Victoria Village'

# Let's Explore top 100 venue that are in toronto within 1000 radius

first, let's create get request URL. name my URL is 'url'

In [16]:
latitude=43.6534817 #  latitude of toronto
longitude=-79.3839347 # longitude of toronto
radius=1000
LIMIT=100
url='http://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
CLIENT_ID,
CLIENT_SECRET,
VERSION,
latitude,
longitude,
radius,
LIMIT)

In [17]:
url

'http://api.foursquare.com/v2/venues/explore?&client_id=PVFHMYLB5SNS10OVXLYOKRKT0YFJWLCZUSUH0CARQO4QMB3N&client_secret=TROKPKIXHV2EQJUDC32KDANASYMXNZ1MZMLHC30H5TWU3SKL&v=20210705&ll=43.6534817,-79.3839347&radius=1000&limit=100'

In [19]:
import requests
import json
result=requests.get(url).json()
result

{'meta': {'code': 200, 'requestId': '60e28271c3c75a6f8bf7a976'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 238,
  'suggestedBounds': {'ne': {'lat': 43.66248170900001,
    'lng': -79.37151886118865},
   'sw': {'lat': 43.64448169099999, 'lng': -79.39635053881135}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5227bb01498e17bf485e6202',
       'name': 'Downtown Toronto',
       'location': {'lat': 43.65323167517444,
        'lng': -79.38529600606677,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.65323167517444,
  

In [20]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [21]:
from pandas.io.json import json_normalize
venues = result['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
# What does apply function in python
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

C:\Users\ASUS\downloder\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Nathan Phillips Square,Plaza,43.652270,-79.383516
2,LUSH,Cosmetics Shop,43.653557,-79.380400
3,M Square Coffee Co,Coffee Shop,43.651218,-79.383555
4,Elgin And Winter Garden Theatres,Theater,43.653394,-79.378507


In [22]:
nearby_venues

,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Nathan Phillips Square,Plaza,43.652270,-79.383516
2,LUSH,Cosmetics Shop,43.653557,-79.380400
3,M Square Coffee Co,Coffee Shop,43.651218,-79.383555
4,Elgin And Winter Garden Theatres,Theater,43.653394,-79.378507
...,...,...,...,...
95,Hudson's Bay,Department Store,43.652040,-79.380391
96,Jimmy's Coffee,Café,43.655827,-79.392042
97,Craft Beer Market,Beer Bar,43.649890,-79.378005
98,Tim Hortons,Coffee Shop,43.658570,-79.385123


In [23]:
nearby_venues_gc=nearby_venues.groupby('categories').count()# group the dataframe based on categories 
nearby_venues_gc.reset_index(inplace=True)# reset index
nearby_venues_gc.drop(columns=['lat','lng'])# drop the lat, lng column

,categories,name
0,American Restaurant,2
1,Art Gallery,2
2,Art Museum,1
3,Arts & Crafts Store,1
4,Asian Restaurant,1
5,Bakery,1
6,Beer Bar,2
7,Bookstore,1
8,Brazilian Restaurant,1
9,Breakfast Spot,1
